In [2]:
import pymongo

# 连接MongoDB
client = pymongo.MongoClient(host='localhost', port=27017)
client = pymongo.MongoClient('mongodb://localhost:27017/')
# 指定数据库
db = client.spider
db = client['spider']
# 指定集合
collection = db.students
collection = db['students']

In [3]:
# 插入单条数据
student = {
    'id': '20170101',
    'name': 'Jordan',
    'age': 20,
    'gender': 'male'
}
result = collection.insert_one(student)
print(result.inserted_id)

6065edc6d9635fe105852195


In [4]:
# 插入多条数据
student1 = {
    'id': '20170101',
    'name': 'Jordan',
    'age': 20,
    'gender': 'male'
}

student2 = {
    'id': '20170202',
    'name': 'Mike',
    'age': 21,
    'gender': 'male'
}

result = collection.insert_many([student1, student2])
print(result.inserted_ids)
print(result.inserted_ids[0])

[ObjectId('6065edc6d9635fe105852196'), ObjectId('6065edc6d9635fe105852197')]
6065edc6d9635fe105852196


In [5]:
# 查询单个结果

result = collection.find_one({'name': 'Mike'})
print(type(result))
print(result)

<class 'dict'>
{'_id': ObjectId('606549c7f9d122c63b3f2824'), 'id': '20170202', 'name': 'Mike', 'age': 25, 'gender': 'male'}


In [6]:
from bson.objectid import ObjectId

# 用_id查询
result = collection.find_one({'_id': ObjectId('606549c7f9d122c63b3f2824')})
print(result)

{'_id': ObjectId('606549c7f9d122c63b3f2824'), 'id': '20170202', 'name': 'Mike', 'age': 25, 'gender': 'male'}


In [7]:
# 查询所有结果
results = collection.find({'age': 20})
print(results)
print(type(results))
for result in results:
    print(result)

<class 'pymongo.cursor.Cursor'>
{'_id': ObjectId('6065edc6d9635fe105852195'), 'id': '20170101', 'name': 'Jordan', 'age': 20, 'gender': 'male'}
{'_id': ObjectId('6065edc6d9635fe105852196'), 'id': '20170101', 'name': 'Jordan', 'age': 20, 'gender': 'male'}


In [8]:
复杂查询条件, 条件为一个字典

$lt 小于 {'age': {'$lt': 20}}
$gt 大于 {'age': {'$gt': 20}}
$lte 小于等于 {'age': {'$lte': 20}}
$gte 大于等于 {'age': {'$gte': 20}}
$ne  不等于 {'age': {'$ne': 20}}
$in 在范围内 {'age': {'$in': [20, 23]}}
$nin 不在范围内 {'age': {'$nin': [20, 23]}}
$regex name以M开头  {'name': {'$regex': '^M.*'}}
$exists 属性是否存在 {'name': {'$exists': True}} (name 属性存在)
$type 类型判断 {'age': {'$type': 'int'}} (age 的类型为 int)
$mod 数字模操作 {'age': {'$mod': [5, 0]}} (年龄模 5 余 0)
$text 文本查询 {'$text': {'$search': 'Mike'}} (text 类型的属性中包含 Mike 字符串)
$where 高级条件查询 {'$where': 'obj.fans_count == obj.follows_count'} (自身粉丝数等于关注数)
其中 obj 可换为 this  

https://docs.mongodb.com/manual/reference/operator/query/

SyntaxError: invalid syntax (<ipython-input-8-59e478753233>, line 3)

In [9]:
# 计数

# 用cursor.count()
count = collection.find().count()
print(count)
count = collection.find({'age': 20}).count()
print(count)
print()

# 用db.collection.countDocuments(query)
count = collection.count_documents({})
print(count)
count = collection.count_documents({'age': 20})
print(count)
print()

5
2

5
2

ipykernel_launcher:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
ipykernel_launcher:6: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


In [11]:
# 排序

results = collection.find().sort('name', pymongo.ASCENDING)
print([result['name'] for result in results])

['Jordan', 'Jordan', 'Jordan', 'Mike', 'Mike']


In [12]:
# 偏移

# skip(n) 选择忽略前面n个元素
results = collection.find().sort('name', pymongo.ASCENDING).skip(0)
print([result['name'] for result in results])

# limit(n) 只取n个结果
results = collection.find().sort('name', pymongo.ASCENDING).skip(0).limit(1)
print([result['name'] for result in results])

# 值得注意的是，在数据库数量非常庞大的时候，如千万、亿级别，最好不要使用大的偏移量来查询数据，因为这样很可能导致内存溢出。这时可以使用类似如下操作来查询：
# from bson.objectid import ObjectId
# collection.find({'_id': {'$gt': ObjectId('593278c815c2602678bb2b8d')}})

['Jordan', 'Jordan', 'Jordan', 'Mike', 'Mike']
['Jordan']


In [24]:
# 更新

condition = {'name': 'Jordan'}
student = collection.find_one(condition)
student['age'] = 25

# 整体更新, 
# result = collection.update(condition, student)
# print(result)

# 字段更新
# result = collection.update(condition, {'$set': student})
# print(result)

# update_one 
student['age'] = 20
result = collection.update_one(condition, {'$set': student})
print(result)
print(result.matched_count, result.modified_count)

condition = {'age': {'$gt': 20}}
result = collection.update_one(condition, {'$inc': {'age': 1}})
print(result)
print(result.matched_count, result.modified_count)

# update_many
condition = {'age': {'$gt': 20}}
result = collection.update_many(condition, {'$inc': {'age': 1}})
print(result)
print(result.matched_count, result.modified_count)

1 0
1 1
2 2


In [32]:
# 删除

# result = collection.remove({'name': 'Kevin'})
# print(result)

# delete_one() 和 delete_many()
result = collection.delete_one({'name': 'Kevin'})
print(result)
print(result.raw_result)
print(result.deleted_count)

result = collection.delete_many({'age': {'$lt': 25}})
print(result.deleted_count)

{'n': 0, 'ok': 1.0}
0
0
